Ignis: Latent Dirichlet Allocation
============

In [1]:
import ignis

Model training (LDA)
----

Load from an `ignis.Corpus`, add the processed docs to an LDA model, and train it.

The random seed and parallelisation can both affect results, so setting the seed and number of workers is necessary for reproducibility.

(Ignis sets both of them to an initial default value to attempt to maintain reproducibility, but these defaults can be overridden as necessary.)

In [39]:
corpus = ignis.load_corpus("bbc.corpus")

In [3]:
model_options = {
    "term_weighting": "pmi",
    "k": 5,
    "until_max_coherence": True,
    "verbose": True,
    "alpha": 0.5,
    "eta": 0.005
}

In [4]:
results = ignis.train_model(corpus, model_options=model_options,)
results.init_vis("pyldavis", verbose=True)
results.show_visualisation()

Training model on 2118 documents:
{'term_weighting': 'pmi', 'k': 5, 'seed': 11399, 'workers': 8, 'parallel_scheme': 'default', 'iterations': 2000, 'update_every': 100, 'until_max_ll': False, 'until_max_coherence': True, 'max_extra_iterations': 3000, 'verbose': True, 'alpha': 0.5, 'eta': 0.005, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}





Continuing to train until maximum coherence.




Best coherence: -1.93574 (Starting: -2.13439)
Docs: 2118, Vocab size: 19290, Total Words: 412636
Model training complete. (51.311s)
Preparing LDA visualisation... Done. (0.355s)


In [5]:
# results.export_visualisation("bbc_results")

## Saving results, using the automated labeller

In [5]:
results.save("bbc.aurum")
results = ignis.load_results("bbc.aurum")

In [6]:
results.init_labeller("tomotopy", verbose=True)

Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.


## Exploring documents that "belong" to a given topic

Because topics are distributions over words and documents are *distributions* over topics, documents don't belong to individual topics per se; every topic is represented in every document with some probability.

We therefore have to specify how many of the document's top `n` topics we want to check for the actual topic we're interested in.

This is especially significant for topic models that use a term weighting scheme, because all the common words (i.e., what we might consider stopwords) tend to get grouped into a single large topic; if we only consider each document's single most probable topic, we will unintentionally exclude documents which have this "stopwords" topic as their top topic.  (This is, however, not always a given; one would need to check the model's output and iterate from there).

Alternatively, we can specify `ignore_topics` when slicing to completely ignore a given topic when classifying documents.

In [7]:
results.nb_explore_topics()

interactive(children=(IntSlider(value=1, description='Topic', max=5, min=1), Output()), _dom_classes=('widget-…

<function ignis.aurum.Aurum.nb_explore_topics.<locals>.show_topic(topic_id=1)>

Slicing and iteration
--------
After seeing what the main topics might be, we can slice the initial corpus further and re-run topic modelling to get better resolution.

In [8]:
# Try zooming in on a sub-topic
tech_slice = results.slice_by_topic(topic_id=1)

We might start by checking exactly which documents a slice of the corpus contains, by exploring it directly:

In [9]:
# If all the documents in this slice have the same top topic and we order them by the probability of that topic descending,
# we will get the same document order as in the `nb_explore_topics()` widget above.
tech_slice.nb_explore(doc_sort_key=lambda doc: results.get_document_top_topic(doc.id)[1], reverse=True)

interactive(children=(IntSlider(value=0, description='Document', max=335), Output()), _dom_classes=('widget-in…

<function ignis.corpus.CorpusSlice.nb_explore.<locals>.show_doc(index=0)>

Ignis can suggest a recommended number of topics to use for LDA, based on the coherence scores of a range of lightly-trained models.

The number of words per topic to consider when calculating coherence (`top_n`) roughly affects the number of suggested topics:

A higher `top_n` tends to result in a lower suggested topic count, and a lower `top_n` tends to result in a higher suggested topic count.  The default of `30` corresponds to the number of top words shown by pyLDAvis.

In [15]:
# Getting a suggested number of topics and retraining the model on the new slice
best_k = results.resuggest_num_topics(
    corpus_slice=results.slice_by_topic(topic_id=1),
    verbose=True,
    iterations=100
)

Training 9 mini-models to suggest a suitable number of topics between 2 and 10.
(336 documents, 100 iterations each, considering top 30 terms per topic)


Suggested topic count: 3	Coherence: -2.97021 (always negative, closer to 0 is better)
All suggestions: [3] -2.97021, [2] -3.47360, [5] -3.56812, [6] -3.99610, [4] -4.31655, [10] -4.48340, [8] -4.52375, [7] -4.82705, [9] -5.21306



In [ ]:
entertainment_model = results.retrain_model(
    corpus_slice=entertainment_slice, model_options={"k": best_k}
)
entertainment_model.show_visualisation()

The position of each topic cluster on the graph is not intrinsically informative per se (being simply the result of some specified dimensionality-reducing technique), but if we want we can run the modelling algorithm with a different random seed and see if we get a more meaningful set of topics.

(We can also override any of the previously set options.)

In [13]:
new_model_options = {"seed": 7156}
entertainment_model_2 = entertainment_model.retrain_model(
    model_options=new_model_options
)
entertainment_model_2.show_visualisation()

Training model on 332 documents:
{'term_weighting': 'pmi', 'k': 3, 'seed': 7156, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': False, 'until_max_coherence': True, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}





Continuing to train until maximum coherence.




Best coherence: -2.68437 (Starting: -2.68437)
Docs: 332, Vocab size: 8667, Total Words: 54872
Model training complete. (4.213s)
Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.


Notice that the topic words remain more or less consistent across different training runs, even though their positions in the visualisation change with each iteration.

In [14]:
entertainment_model_2.nb_explore_topics()

interactive(children=(IntSlider(value=1, description='Topic', max=3, min=1), Output()), _dom_classes=('widget-…

<function ignis.aurum.Aurum.nb_explore_topics.<locals>.show_topic(topic_id=1)>

In addition to simply slicing by topic, we can also explicitly search the whole corpus for documents that contain certain tokens, in case we want to be absolutely sure we got all the documents that mention certain words or phrases.

In [15]:
# Topics that seem to deal with films
film_topic = entertainment_model_2.slice_by_topics(topic_ids=[4, 5])

# Tokens that are related to films, doing a full-text search through the entire corpus (not just within the current results)
by_token_slice = entertainment_model_2.slice_by_tokens(
    ["film", "films", "movie", "cinema"], include_root=True
)

concat_slice = film_topic.concat(by_token_slice)

concat_model = entertainment_model_2.retrain_model(
    concat_slice, model_options=model_options
)
concat_model.show_visualisation()

Training model on 281 documents:
{'term_weighting': 'pmi', 'k': 3, 'seed': 7156, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': False, 'until_max_coherence': True, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}





Continuing to train until maximum coherence.




Best coherence: -1.77912 (Starting: -1.77912)
Docs: 281, Vocab size: 9285, Total Words: 58688
Model training complete. (4.430s)
Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.


In [16]:
# Topic 2 seems related to film awards
concat_model.nb_explore_topics()

interactive(children=(IntSlider(value=1, description='Topic', max=3, min=1), Output()), _dom_classes=('widget-…

<function ignis.aurum.Aurum.nb_explore_topics.<locals>.show_topic(topic_id=1)>